In [24]:
from spdm.data.sp_property import sp_tree, sp_property, SpTree
from spdm.data.HTree import HTree, List, Dict
from spdm.data.Entry import open_entry
from spdm.data.AoS import AoS
import typing
import pprint

__getitem__和__getattr__都是Python中的特殊方法，用于访问对象的属性。它们的区别在于：

__getitem__用于访问对象的索引，例如obj[key]，其中key是索引值。
__getattr__用于访问对象的属性，例如obj.attr，其中attr是属性名。
如果对象同时实现了__getitem__和__getattr__，那么在访问对象的属性时，Python会先调用__getattr__方法，如果该方法抛出AttributeError异常，则会调用__getitem__方法。

In [14]:
### Python中的内置函数，用来返回、修改、删除一个对象的属性

from typing import Any


class Foo:
    def __getitem__(self, key: str) -> typing.Any:
        print(f"__getitem__({key})")

    def __setitem__(self, key: str, value: typing.Any):
        print(f"__setitem__({key}, {value})")

    def __delitem__(self, key: str):
        print(f"__delitem__({key})")

    def __getattr__(self, __name: str) -> Any:
        print(f"__getattr__({__name})")

    def __setattr__(self, __name: str,value) -> Any:
        print(f"__setattr__({__name})")
        
    def __delattr__(self, __name: str) -> Any:
        print(f"__delattr__({__name})")
        

In [15]:
foo=Foo()

foo["a"] 

__getitem__(a)


In [11]:
foo["a"]=1

__setitem__(a, 1)


In [8]:
del foo["a"]

__delitem__(a)


In [12]:
foo.a

__getattr__(a)


In [16]:
foo.a=5

__setattr__(a)


In [17]:
del foo.a

__delattr__(a)


sp_tree是一个装饰器，将一个类转换为 SpTree 类
SpTree类上赋予了很多额外的功能。为什么要转化为SpTree类

怎么叙述这件事情，首先表述清楚，数据绑定的作用是什么
IMAS DD将托克马克描述规整成层次化的树状结构，为实验和模拟提供了统一的语义标准。
数据是集成建模分析的基础，方便的处理数据是集成框架的核心功能，特别是适用于层次化树状结构的语义数据的处理。
在前面叙述的SpDB的功能中，可以看到，Entry及Open_Entry两个标准的API，可以处理多种不同类似的科学数据，并且提供指针漂移的方法，灵活获取需要的数据。
但是，。。。，需要将获取的数据和IMAS DD的语义进行绑定。
那为什么SpDB还要进行翻译，比如直接MDS的数据，psi值，
所以，这里有两层的翻译
- 格式的转化
- 结构的翻译（映射） 如果少了这一步呢？这一步的目的是方便后续语义的绑定
- 语义（数据）的绑定
- 数据绑定的目的是什么？（这个之前实际上是通用的，不依赖于具体的语义）
但是对于tokmak，IMAS DD定义了一套标准的语义，
递归地为子节点的属性分配数据
不同层次的数据绑定
- python中原始本地数据类型自称的树状结构


将相关物理量组织成层次化的结构，实现物理量的自洽、自动演化。这既保证了物理量之间的关系的唯一确定性，又简化了计算模型的构建过程

而IDS除了描述了装置的子系统，还描述了物理概念，这些物理概念通常由单个物理量、简单固定的物理计算公式及复杂的模拟程序所决定及组成。
对应一个或者多个模拟程序。


本来在裸数据里面是字典，没有功能，很多量也没有，特别是导出来
通过绑定，将导出来在需要的时候再计算，保证计算结果的一致性。关系是function保证的。保证结果的唯一性。

绑定让数据赋予了功能

IMAS 标称属性转化为。。。。

按照属性访问
类型提示，将裸数据转化为具有功能的对象实体（数组转化为表达式或者function）

这一块怎么写，分成不同的层次：
- 普通字典
- 自定义数据类型
- 定义函数，自定义计算

#### 原生Python中字典的访问(树状结构的访问)

In [117]:
### 原生Python中字典的访问(树状结构的访问)
data = {
        "name": "Alice",
        "age": 25,
        "hobbies": ["reading", "painting", "yoga"],
        "address": [
            {"street": "123 Main St", "city": "Anytown", "state": "CA", "zip": "12345"},
            {
                "street": "456 Oak St",
                "city": "Othertown",
                "state": "NY",
                "zip": "67890",
            },
            {"street": "789 Elm St", "city": "Somewhere", "state": "CO", "zip": "24680"},
        ],
    }
    # _entry=open_entry(....)

In [68]:
data["name"]


'Alice'

In [69]:
data["age"]

25

In [118]:
data["address"]

[{'street': '123 Main St', 'city': 'Anytown', 'state': 'CA', 'zip': '12345'},
 {'street': '456 Oak St', 'city': 'Othertown', 'state': 'NY', 'zip': '67890'},
 {'street': '789 Elm St', 'city': 'Somewhere', 'state': 'CO', 'zip': '24680'}]

In [89]:
data["address"][0]

{'street': '123 Main St', 'city': 'Anytown', 'state': 'CA', 'zip': '12345'}

#### 将Python中原生数据映射到SpTree中

In [119]:
### 利用sp_tree装饰器定义Data类，Data的输入是一个字典，输出是一个树状结构
@sp_tree
class Data:
    def __init__(self, *args, **kwargs) -> None:
        super().__init__(*args, **kwargs)
### 传递一个普通的字典结构给Data
data = Data(
    {
        "name": "Alice",
        "age": 25,
        "hobbies": ["reading", "painting", "yoga"],
        "address": [
            {"street": "123 Main St", "city": "Anytown", "state": "CA", "zip": "12345"},
            {
                "street": "456 Oak St",
                "city": "Othertown",
                "state": "NY",
                "zip": "67890",
            },
            {"street": "789 Elm St", "city": "Somewhere", "state": "CO", "zip": "24680"},
        ],
    },
    # _entry=open_entry(....)
)

In [91]:
## 访问字典中的数据
data.get("name")

'Alice'

In [120]:
data.get("address")

[{'street': '123 Main St', 'city': 'Anytown', 'state': 'CA', 'zip': '12345'},
 {'street': '456 Oak St', 'city': 'Othertown', 'state': 'NY', 'zip': '67890'},
 {'street': '789 Elm St', 'city': 'Somewhere', 'state': 'CO', 'zip': '24680'}]

In [96]:
### 但是address不支持直接切片访问。
data.get("address[1]")

<tags.not_found: 0>

In [121]:
### address只能以整体拿回来，然后再切片访问
data.get("address")[1]

{'street': '456 Oak St', 'city': 'Othertown', 'state': 'NY', 'zip': '67890'}

#### sp_tree装饰器支持在Data类中声明字典中的每个元素key及其value的数据类型

In [122]:
### sp_tree装饰器支持在Data类中声明字典中的每个元素key及其value的数据类型，这样就可以直接访问字典中的元素。
@sp_tree
class Data:
    def __init__(self, *args, **kwargs) -> None:
        super().__init__(*args, **kwargs)

    name: str
    age: int
    hobbies: List[str]
    address: List[Dict]   
    # address: AoS[Dict]
    # address: AoS[Addrees]


data = Data(
    {
        "name": "Alice",
        "age": '25',
        "hobbies": ["reading", "painting", "yoga"],
        "address": [
            {"street": "123 Main St", "city": "Anytown", "state": "CA", "zip": "12345"},
            {
                "street": "456 Oak St",
                "city": "Othertown",
                "state": "NY",
                "zip": "67890",
            },
            {"street": "789 Elm St", "city": "Somewhere", "state": "CO", "zip": "24680"},
        ],
    },
    # _entry=open_entry(....)
)

In [108]:
## 发现声明过的元素其颜色都是蓝色的
data.name

'Alice'

In [111]:
### 除了声明元素的key值，同时可以强制转化原有数据的数据类型
type(data.age)

int

In [ ]:
### 为什么数据在_cache里面
data.address

[{'street': '123 Main St', 'city': 'Anytown', 'state': 'CA', 'zip': '12345'},
 {'street': '456 Oak St', 'city': 'Othertown', 'state': 'NY', 'zip': '67890'},
 {'street': '789 Elm St', 'city': 'Somewhere', 'state': 'CO', 'zip': '24680'}]

In [62]:
## zip颜色是灰色，因为Dict里面的元素没有一一声明，
data.address[0].zip

AttributeError: 'Dict' object has no attribute 'zip'

In [112]:
### 支持对LIST元素的直接切片访问，而不需要拿回整个字典
data.address[0]._cache

{'street': '123 Main St', 'city': 'Anytown', 'state': 'CA', 'zip': '12345'}

### 进一步定义复杂AOS内的数据结构

In [129]:
### 增加Address类，定义address的数据类型

@sp_tree
class Addrees:
    street: str
    city: str
    state: str
    zip: int


@sp_tree
class Data:
    def __init__(self, *args, **kwargs) -> None:
        super().__init__(*args, **kwargs)

    name: str
    age: int
    hobbies: List[str]
    address: AoS[Addrees]


data = Data(
    {
        "name": "Alice",
        "age": '25',
        "hobbies": ["reading", "painting", "yoga"],
        "address": [
            {"street": "123 Main St", "city": "Anytown", "state": "CA", "zip": "12345"},
            {
                "street": "456 Oak St",
                "city": "Othertown",
                "state": "NY",
                "zip": "67890",
            },
            {"street": "789 Elm St", "city": "Somewhere", "state": "CO", "zip": "24680"},
        ],
    },
    # _entry=open_entry(....)
)

In [130]:
### zip的颜色变成了蓝色，因为在Addrees类中声明了zip
data.address[0].zip

12345

In [132]:
data.address[1].street

'456 Oak St'

In [131]:
data.address[0]._cache

{'street': '123 Main St', 'city': 'Anytown', 'state': 'CA', 'zip': 12345}

### 直接增加一个元素

In [134]:


@sp_tree
class Addrees:
    street: str
    city: str
    state: str
    zip: int
    building: str = "#12345" # default value
    neighbour: str


@sp_tree
class Data:
    def __init__(self, *args, **kwargs) -> None:
        super().__init__(*args, **kwargs)

    name: str
    age: int
    hobbies: List[str]
    address: AoS[Addrees]


data = Data(
    {
        "name": "Alice",
        "age": 25,
        "hobbies": ["reading", "painting", "yoga"],
        "address": [
            {"street": "123 Main St", "city": "Anytown", "state": "CA", "zip": "12345"},
            {
                "street": "456 Oak St",
                "city": "Othertown",
                "state": "NY",
                "zip": "67890",
            },
            {"street": "789 Elm St", "city": "Somewhere", "state": "CO", "zip": "24680"},
        ],
    },
    # _entry=open_entry(....)
)



In [135]:
data.address[0].building

'#12345'

In [137]:
data.address[0].neighbour="zhangshan"

In [144]:
### 应用才会生效
data.address[1].building

'#12345'

In [139]:
data.address[0].neighbour

'zhangshan'

In [146]:
data.address[1].neighbour = "lisi"

In [142]:
data.address[0]._cache

{'street': '123 Main St',
 'city': 'Anytown',
 'state': 'CA',
 'zip': '12345',
 'building': '#12345',
 'neighbour': 'zhangshan'}

In [147]:
data.address[1].neighbour

{'street': '456 Oak St',
 'city': 'Othertown',
 'state': 'NY',
 'zip': '67890',
 'building': '#12345',
 'neighbour': 'lisi'}

### 高阶功能，增加函数sp_property，定义函数

In [149]:
@sp_tree
class House:
    def __init__(self, *args, **kwargs) -> None:
        super().__init__(*args, **kwargs)
        pprint.pprint((args, kwargs))

    level: int = 4
    length: float
    width: float

    @sp_property(units="m^2")
    def area(self) -> float:
        return self.width * self.length


@sp_tree
class Addrees:
    street: str
    city: str
    state: str
    zip: int
    building: str = "#12345"
    neighbour: str
    house: House = sp_property(label="big house", default_value={"level": 2, "area": 1000})


@sp_tree
class Data:
    def __init__(self, *args, **kwargs) -> None:
        super().__init__(*args, **kwargs)

    name: str
    age: int
    hobbies: List[str]
    address: AoS[Addrees]


data = Data(
    {
        "name": "Alice",
        "age": 25,
        "hobbies": ["reading", "painting", "yoga"],
        "address": [
            {"street": "123 Main St", "city": "Anytown", "state": "CA", "zip": "12345"},
            {
                "street": "456 Oak St",
                "city": "Othertown",
                "state": "NY",
                "zip": "67890",
                "house": {"length": 5, "width": 10},
            },
            {"street": "789 Elm St", "city": "Somewhere", "state": "CO", "zip": "24680"},
        ],
    },
    # _entry=open_entry(....)
)



In [150]:
house=data.address[1].house

(({'length': 5, 'width': 10},),
 {'_entry': None,
  '_parent': <__main__.Addrees object at 0x7efc7ea7f760>,
  'default_value': {'area': 1000, 'level': 2},
  'label': 'big house',
  'name': 'house'})


In [151]:
house.level

4

In [152]:
house.length

5.0

In [153]:
house.width

10.0

In [154]:
house.area

50.0

In [155]:
House.area.metadata.get("units")

'm^2'

In [156]:
data.get("address/3/street", "I don't know")

"I don't know"

In [157]:
for address in data.address:
    print(address.street)

123 Main St
456 Oak St
789 Elm St
tags.not_found


In [158]:
data.address[0].street = "456 Main St"

In [159]:
data.address[0].street

'456 Main St'

In [ ]:
data2 = Data({}, _entry=open_entry("file://./data/address.json"))

In [ ]:

file_entry可以给data，这个明天试下。
file_entry = open_entry(f"file+geqdsk://{workdir}//data/g900003.00230_ITER_15MA_eqdsk16HR.txt/#equilibrium")

In [2]:



class Foo(SpTree):
    boo: Data = sp_property(label="boo", default_value=1.0)


class Bar(Foo):
    boo: Data = sp_property(label="boo", units="m")


@sp_tree
class Bar2(Foo):
    boo: Data = 2.1234


a = Bar()
b = Bar2()
print(a.boo._metadata)
print(b.boo._metadata)
print(b.get("boo")._metadata)

{'label': 'boo', 'name': 'boo', 'units': 'm'}
{'label': 'boo', 'name': 'boo'}
{'label': 'boo', 'name': 'boo'}
